In [1]:
import re
from datetime import datetime
import pandas as pd
import glob,os
import sys 
import zipfile
import numpy as np
import spacy



# Funciones 

In [2]:
def path_recent_chat(Carpeta_path: str):
    """ Devuelve el df de la hoja especifica, del archivo mas reciente modificado de la carpeta especificada"""
    Path_n= Carpeta_path + '/*'
    tipo_de_archivo = '*.txt'
    # Busca el archivo más reciente
    archivos = glob.glob(Path_n + tipo_de_archivo)
    archivo_mas_reciente = max(archivos, key=os.path.getmtime)
    nombre_del_archivo_N = os.path.basename(archivo_mas_reciente)
    print(archivo_mas_reciente)
    return archivo_mas_reciente

def split_chat_messages(raw_text): 
    ''' funcion que splitea mensajes individuales de acuerdo al formate (iphone/android)'''
    # Verificar si es formato iPhone (comienza con corchetes)
    raw_text = raw_text.replace('\u200e', '').replace('\u202f', ' ').replace('\u00a0', ' ')

    iphone_format= False
    if re.search(r'\[\d{1,2}/\d{1,2}/\d{2}, \d{1,2}:\d{2}:\d{2} p\. m\.\]', raw_text):
        print('match iphone')
        # Formato iPhone
        processed_text = raw_text.replace('[', '\n[', 1).replace('\n[', '[', 1)
        iphone_format = True
        return re.split(r'\n(?=\[)', processed_text.strip()),iphone_format
    else:
        # Formato Android
        print('match android')
        android_pattern = r'\n(?=\d{1,2}/\d{1,2}/\d{4}, \d{1,2}:\d{2} - )'
        iphone_format = False
        return re.split(android_pattern, raw_text.strip()),iphone_format    
def parse_chat_line(line, format_flag=None):
    '''Función que separa cada mensaje individual en 3 grupos (timestamp, sender, message)'''
    # Eliminar caracteres especiales y espacios no deseados
    clean_line = line.replace('\u200e', '').replace('\u202f', ' ').replace('\u00a0', ' ').strip()
    
    if not clean_line:
        return None
    
    # Patrones para ambos formatos
    pattern_iphone = r'\[(\d{1,2}/\d{1,2}/\d{2,4}),\s*(\d{1,2}:\d{2}:\d{2}\s*(?:a\. m\.|p\. m\.))\]\s*(.*?):\s+(.+)'
    pattern_android = r'(\d{1,2}/\d{1,2}/\d{2,4},\s*\d{1,2}:\d{2})\s*-\s*(.*?):\s*(.*)'
    
    # Patrón para mensajes del sistema (sin remitente)
    pattern_system = r'(\d{1,2}/\d{1,2}/\d{2,4},\s*\d{1,2}:\d{2})\s*-\s*(.*)'

    # Intentar con iPhone primero
    match = re.match(pattern_iphone, clean_line,flags=re.DOTALL)
    if match:
        date_str, time_str, sender, message = match.groups()
        # Unificar fecha y hora en objeto datetime
        dt_str = f"{date_str} {time_str}".replace("a. m.", "AM").replace("p. m.", "PM")
        return (dt_str, sender, message)
    
    # Intentar con Android (incluyendo mensajes vacíos)
    match = re.match(pattern_android, clean_line,flags=re.DOTALL)
    if match:
        timestamp, sender, message = match.groups()
        # Si el mensaje está vacío, asignar cadena vacía explícitamente
        if message is None:
            message = ""
        return (timestamp, sender, message)
    
    # Intentar con mensajes del sistema (sin remitente)
    match = re.match(pattern_system, clean_line,flags=re.DOTALL)
    if match:
        timestamp, message = match.groups()
        return (timestamp, "Sistema", message)
    
    # Si coincide con el patron formateado:
    
    return None

def extraer_linea_completa(df):
    # Usamos una lista de comprensión con regex por fila (más eficiente que apply)
    resultados = []
    for valor, texto in zip(df['codigos_inc'], df['message_first']):
        patron = re.compile(rf"^.*{re.escape(valor)}.*$", re.MULTILINE)
        coincidencia = patron.search(texto)
        resultados.append(coincidencia.group(0).strip() if coincidencia else None)
    return resultados

def extraer_bloque_por_inc(df):
    resultados = []
    for valor, texto in zip(df['codigos_inc'], df['message']):
        if pd.isna(valor) or pd.isna(texto):
            resultados.append(None)
            continue
        valor_escapado = re.escape(valor)
        # Buscar la posición del código INC específico
        inc_match = re.search(valor_escapado, texto)
        if not inc_match:
            resultados.append(None)
            continue  
        inc_pos = inc_match.start()
        texto_despues_inc = texto[inc_pos:]
        
        # Encontrar todas las ocurrencias de HI después de este INC
        hi_ocurrencias = list(re.finditer(r'\bHI\b', texto_despues_inc, re.IGNORECASE))
        
        if len(hi_ocurrencias) >= 2:
            # Cortar hasta la segunda HI
            segunda_hi_pos = hi_ocurrencias[1].start()
            bloque_final = texto_despues_inc[:segunda_hi_pos].strip()
        else:
            # Si no hay segunda HI, tomar TODO el texto desde el INC actual
            bloque_final = texto_despues_inc.strip()
        
        # Limpiar tabs y espacios extras
        bloque_final = re.sub(r'\t', ' ', bloque_final)
        bloque_final = re.sub(r'\n{3,}', '\n\n', bloque_final)
        
        resultados.append(bloque_final)
            
    return resultados


In [3]:
if len(sys.argv) > 1: ## Ruta que le pasa el watchdog
    ruta_archv = sys.argv[1]
else:
    print('No se paso ruta de archivo.')

In [4]:
#ruta_archv = r'D:\DATA\wpp\Chat de WhatsApp con Averias Norte-VOC-CICSA.zip'
#ruta_archv = r'D:\DATA\wpp\test_format.zip'


In [5]:

with zipfile.ZipFile(ruta_archv, 'r') as zipf:
    # Listar todos los archivos
    print("Archivos en el ZIP:")
    for file_info in zipf.infolist():
        print(f"Nombre: {file_info.filename}")
        print(f"Tamaño: {file_info.file_size} bytes")
        print(f"Comprimido: {file_info.compress_size} bytes")
        print("---")
    
    # Leer contenido de un archivo específico
    with zipf.open(zipf.filelist[0].filename) as file:
        raw_text = file.read().decode('utf-8')


Archivos en el ZIP:
Nombre: Chat de WhatsApp con Averias Norte-VOC-CICSA.txt
Tamaño: 1030317 bytes
Comprimido: 91001 bytes
---


In [7]:
# Leer todo el archivo como un solo string
data = []

# Limpiar caracteres invisibles
raw_text = raw_text.replace('\u200e', '').replace('\u202f', ' ').replace('\u00a0', ' ')

# Añadir marcador para luego reponer los corchetes
#raw_text = raw_text.replace('[', '\n[', 1).replace('\n[', '[', 1)  # Evita dividir antes del primer mensaje
messages,format_flag = split_chat_messages(raw_text)  # Split en cada línea que empieza con [

print(f"Total mensajes detectados: {len(messages)}")

match android
Total mensajes detectados: 4738


In [9]:
# ruta_archv = path_recent_chat(r'D:\DATA\wpp')
#with open(ruta_archv, "r", encoding="utf-8") as file:
#     raw_text = file.read()

In [10]:
list_msg_split = [] ## el flags indica que formato estoy afrontando
if not format_flag:
    messages = messages[1:]
else:
    pass
for msg in messages:
    result = parse_chat_line(msg)
    if result:
        list_msg_split.append(result)
    else:
        # Manejar líneas que no coinciden con ningún patrón
        list_msg_split.append(("Desconocido", "Desconocido", msg))

# Convertir a DataFrame de pandas
df = pd.DataFrame(list_msg_split, columns=['timestamp', 'sender', 'message'])
df.head()

,timestamp,sender,message
0,"21/7/2025, 09:21",+51 914 684 619,
1,"21/7/2025, 09:21",+51 914 684 619,<Multimedia omitido>
2,"21/7/2025, 09:21",+51 914 684 619,<Multimedia omitido>
3,"21/7/2025, 09:21",+51 914 684 619,<Multimedia omitido>
4,"21/7/2025, 09:22",+51 914 089 629,<Multimedia omitido>


In [11]:
format_flag

False

In [12]:
if not format_flag: 
    df.timestamp = pd.to_datetime(df.timestamp,format = '%d/%m/%Y, %H:%M')
   
else: 
    df.timestamp = df.timestamp.replace("a. m.", "AM").replace("p. m.", "PM")
    df.timestamp = pd.to_datetime(df.timestamp,format = "%d/%m/%y %I:%M:%S %p")

## df base

In [13]:
df.timestamp.describe()

count                             4737
mean     2025-08-04 08:50:06.333122304
min                2025-07-21 09:21:00
25%                2025-07-26 07:28:00
50%                2025-08-03 10:44:00
75%                2025-08-12 20:28:00
max                2025-08-22 16:26:00
Name: timestamp, dtype: object

In [14]:
### Filtro por solo mensajes de Incidencias

## solo mensajes que contengan numero de Incidendcias
df_inc = df[df.message.str.contains('INC',na=False)].copy()

### Separo el numero de incidencia para usarlo de indice 
df_inc['codigos_inc'] = df_inc['message'].str.findall(r'INC\d{9,}') # columna con codigo de incidencia
df_inc_e = df_inc.explode('codigos_inc').reset_index(drop=True)#Explando las duplicadas




In [15]:
if (df_inc_e[['sender','message']].any()).any():
    print(True)

True


In [18]:
### Solo incidencia Cerradas 
cerrado_df_C = df_inc_e[df_inc_e['message'].str.contains(r'CIERRE|CERRADO', na=False)].sort_values('timestamp').copy()
### Solo la ultima 
cerrado_df_unique = cerrado_df_C.sort_values('timestamp').drop_duplicates('codigos_inc',keep='last') ## me quedo con el ultimo

## Encabezados robustos:

In [19]:
## bajo mi crieterio conviene sacar el encabezado del primer mensaje no del ultimo.
df_inc_e_F = df_inc_e.sort_values('timestamp').drop_duplicates('codigos_inc',keep='first') # primera aparicion
df_merged = pd.merge(cerrado_df_unique,
                     df_inc_e_F[['codigos_inc','message']].rename(columns={'message':'message_first'})
                     ,how='left',on='codigos_inc')

df_merged['Encabezado'] = extraer_linea_completa(df_merged).copy()

df_merged['Encabezado'] = df_merged['Encabezado'].str.replace('\t',' ')


df_merged.drop(['message_first'],axis=1,inplace=True)

df_merged.Encabezado = df_merged.Encabezado.str.replace(r'[_\.]', ' ', regex=True)# quito los '.' '_'
df_merged.Encabezado = df_merged.Encabezado.str.replace(r'\s+', ' ', regex=True).str.strip() # espacios de mas
df_merged.Encabezado = df_merged.Encabezado.str.replace(r' || ', '').str.strip() # espacios de mas



In [20]:
cerrado_df_unique_cut = df_merged.copy()

## Contenido Robusto

In [21]:

# Aplicar la función
cerrado_df_unique_cut['Contenido'] = extraer_bloque_por_inc(cerrado_df_unique_cut)

In [22]:
cerrado_df_unique_cut['Contenido'] = cerrado_df_unique_cut['Contenido'].str.replace(r'(?m)^.*INC0.*\n?', '', regex=True)


cerrado_df_unique_cut['Contenido'] = cerrado_df_unique_cut['Contenido'].str.replace(r'\n{2,}', '\n', regex=True)# quito los saltos de linea de mas 
cerrado_df_unique_cut['Contenido'] = (
    cerrado_df_unique_cut['Contenido']
    .str.replace(r'[ ]{2,}', ' ', regex=True)
    .str.strip()
)

In [23]:
## capturo el formato de INCNNNN (12)valores numericos y todo lo que le sigue hasta mas de 3 /n saltos de linea seguidos.

In [24]:
cerrado_df_unique_cut['Plano'] = cerrado_df_unique_cut.Encabezado.str.split(' ').str[1].str.strip()
cerrado_df_unique_cut['Tipo de Averia'] = cerrado_df_unique_cut.Encabezado.str.split(' ').str[3].str.strip()
cerrado_df_unique_cut['Tipo de Averia'] = cerrado_df_unique_cut['Tipo de Averia'].str.replace(r'[*\+\]]', '', regex=True)
cerrado_df_unique_cut['Nodo'] = cerrado_df_unique_cut.Encabezado.str.split(' ').str[2].str.strip()

In [25]:
cerrado_df_unique_cut['Tipo de Averia'] = cerrado_df_unique_cut['Tipo de Averia'].fillna('PARCIAL')
cerrado_df_unique_cut['Tipo de Averia'] = cerrado_df_unique_cut['Tipo de Averia'].str.replace('¨','')



In [26]:
cerrado_df_unique_cut['Tipo de Averia'] = cerrado_df_unique_cut['Tipo de Averia'].replace({'APRCIAL':'PARCIAL','PACIAL':'PARCIAL'})

In [27]:
cerrado_df_unique_cut.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572 entries, 0 to 571
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   timestamp       572 non-null    datetime64[ns]
 1   sender          572 non-null    object        
 2   message         572 non-null    object        
 3   codigos_inc     572 non-null    object        
 4   Encabezado      572 non-null    object        
 5   Contenido       572 non-null    object        
 6   Plano           572 non-null    object        
 7   Tipo de Averia  572 non-null    object        
 8   Nodo            572 non-null    object        
dtypes: datetime64[ns](1), object(8)
memory usage: 40.3+ KB


In [28]:
cerrado_df_unique_cut

,timestamp,sender,message,codigos_inc,Encabezado,Contenido,Plano,Tipo de Averia,Nodo
0,2025-07-21 09:24:00,+51 914 748 998,*Se actualiza averías (Incidencias y WOs)*\nNO...,INC000009132638,*INC000009132638 TRUJ161 cmtsTrujillo05 TOTAL*,21-07 HI 07:52\nTecnico Marlon Cardenas\nEstim...,TRUJ161,TOTAL,cmtsTrujillo05
1,2025-07-21 09:24:00,+51 914 748 998,*Se actualiza averías (Incidencias y WOs)*\nNO...,INC000009132695,*INC000009132695 TRUJ120 cmtsTrujillo05 PARCIAL*,21-07 HI 07:52\nTecnico Marlon Cardenas\nEstim...,TRUJ120,PARCIAL,cmtsTrujillo05
2,2025-07-21 09:24:00,+51 914 748 998,*Se actualiza averías (Incidencias y WOs)*\nNO...,INC000009132696,*INC000009132696 TRUJ134 cmtsTrujillo05 TOTAL*,21-07 HI 07:52\nTecnico Marlon Cardenas\nEstim...,TRUJ134,TOTAL,cmtsTrujillo05
3,2025-07-21 09:24:00,+51 914 748 998,*Se actualiza averías (Incidencias y WOs)*\nNO...,INC000009132687,*INC000009132687 TRUJ122 vcsTrujillo04 TOTAL*,21-07 HI 07:52\nTecnico Marlon Cardenas\nEstim...,TRUJ122,TOTAL,vcsTrujillo04
4,2025-07-21 09:24:00,+51 914 748 998,*Se actualiza averías (Incidencias y WOs)*\nNO...,INC000009132649,*INC000009132649 TRUJ123 vcsTrujillo03 PARCIAL*,21-07 HI 07:52\nTecnico Marlon Cardenas\nEstim...,TRUJ123,PARCIAL,vcsTrujillo03
...,...,...,...,...,...,...,...,...,...
567,2025-08-22 12:31:00,+51 914 748 998,*INC000009246771 TREP006_cmtsTrujillo04 PARCIA...,INC000009246771,*INC000009246771 TREP006 cmtsTrujillo04 PARCIAL*,HI: 22/8/25 11:40\nTECNICO PEXT: **\nEstimado ...,TREP006,PARCIAL,cmtsTrujillo04
568,2025-08-22 13:06:00,+51 914 748 998,*WO0000002296180 - CORTE DE ENERGIA COMERCIAL*...,INC000009247269,*INC000009247269 TRUJ131 vcsTrujillo03 PARCIAL*,HI: 22/8/25 10:29\nTECNICO PEXT: **\nEstimado ...,TRUJ131,PARCIAL,vcsTrujillo03
569,2025-08-22 15:59:00,+51 989 291 014,*INC000009248766 \tTRLH014_vcsTrujillo04 TOTAL...,INC000009248766,*INC000009248766 TRLH014 vcsTrujillo04 TOTAL*,22/08 HI 15:40\nTecnico **\nse reporta caida t...,TRLH014,TOTAL,vcsTrujillo04
570,2025-08-22 15:59:00,+51 989 291 014,*INC000009248766 \tTRLH014_vcsTrujillo04 TOTAL...,INC000009248758,*INC000009248758 TRLH015-B oltTrujillo06 TOTAL*,22/08 HI 15:40\nTecnico **\nse reporta caida t...,TRLH015-B,TOTAL,oltTrujillo06


In [29]:
cerrado_df_unique_cut['Tipo de Averia'] = np.where(cerrado_df_unique_cut['Tipo de Averia'].isin(['PARCIAL','TOTAL']),
                                                                       cerrado_df_unique_cut['Tipo de Averia'],
                                                                       'OTRO')

In [30]:
cerrado_df_unique_cut['Tipo de Averia'].value_counts()

Tipo de Averia
PARCIAL    340
TOTAL      229
OTRO         3
Name: count, dtype: int64

## Creo Fechas

## Creo Fecha de inicio (robusto con Spacy):

In [31]:
# Procesamiento vectorizado con nlp.pipe
import spacy
from spacy.matcher import Matcher
import unicodedata


def normalizar_texto(texto):
    texto = texto.lower()
    if not any(c in 'áéíóúüñÁÉÍÓÚÜÑ' for c in texto):
        return texto
    return ''.join(c for c in unicodedata.normalize('NFD', texto)
                   if unicodedata.category(c) != 'Mn')


    
nlp = spacy.load("es_core_news_sm")
matcher = Matcher(nlp.vocab)

# Definir el patrón
pattern = [
    {"LEMMA": "tecnico","OP":'+'},
    {"TEXT": "se", "OP" : "?"},
    {"POS" : 'ADP','OP':'?'},
    {"POS" : "DET",'OP':'?'},
    {"LEMMA": {"IN": ["estar", "encontrar", "ir", "llegar", "movilizar", "instalo", "indicar","camino","plano"]}, "OP": "+"},
    {"POS" : 'ADP','OP':'?'},
    {"POS" : "DET",'OP':'?'},
    {"LEMMA": {"IN": ["plano", "ruta", "ubicación"]},"OP":'?'}
]

matcher.add("TECNICO_ACCION", [pattern])    

### test matcher patron 

In [32]:

# Frases de prueba
frases = [
    'tecnico llega al plano',
    'tecnico se moviliza',
    'Técnico llega a plano',
    'Tecnico en camino',
    'tecnico se encuentra en el plano',
    'Técnico camino al plano',
    'tecnico instala',
    'tecnico indica',
    'Tecnico en plano',
    'Tecnico camino a plano',
    'Tecnico en el plano',
    'Técnico en plano',
    'tecnico llega al plano',
    'Tecnico llega al plano',
    'Tecnico atender',
    'Se llega al plano'
]
# Probar coincidencias
for frase in frases:
    texto_normalizado = normalizar_texto(frase)
    print(texto_normalizado)
    print('------------')
    doc = nlp(texto_normalizado)  # Normalizar a minúsculas
    for token in doc:
        print(token.text, token.lemma_)
    matches = matcher(doc)
    if matches:
        print(f":white_check_mark: COINCIDE: {frase}")
    else:
        print(f":x: NO COINCIDE: {frase}")
    print('------------------------')

tecnico llega al plano
------------
tecnico tecnico
llega llegar
al al
plano plano
:white_check_mark: COINCIDE: tecnico llega al plano
------------------------
tecnico se moviliza
------------
tecnico tecnico
se él
moviliza movilizar
:white_check_mark: COINCIDE: tecnico se moviliza
------------------------
tecnico llega a plano
------------
tecnico tecnico
llega llegar
a a
plano plano
:white_check_mark: COINCIDE: Técnico llega a plano
------------------------
tecnico en camino
------------
tecnico tecnico
en en
camino camino
:white_check_mark: COINCIDE: Tecnico en camino
------------------------
tecnico se encuentra en el plano
------------
tecnico tecnico
se él
encuentra encontrar
en en
el el
plano plano
:white_check_mark: COINCIDE: tecnico se encuentra en el plano
------------------------
tecnico camino al plano
------------
tecnico tecnico
camino camino
al al
plano plano
:white_check_mark: COINCIDE: Técnico camino al plano
------------------------
tecnico instala
------------
tecnic

### test in dataframe

In [33]:
def extraer_primer_match_detallado(texto):
    if pd.isna(texto) or not texto.strip():
        return None, None
    for linea in texto.split('\n'):
        linea_n = normalizar_texto(linea)
        doc = nlp(linea_n)
        matches = matcher(doc)
        if matches:
            # Obtener el texto que coincidió exactamente
            span_coincidencia = doc[matches[0][1]:matches[0][2]]
            return linea, span_coincidencia.text
    return None, None


In [34]:
cerrado_df_unique_cut.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572 entries, 0 to 571
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   timestamp       572 non-null    datetime64[ns]
 1   sender          572 non-null    object        
 2   message         572 non-null    object        
 3   codigos_inc     572 non-null    object        
 4   Encabezado      572 non-null    object        
 5   Contenido       572 non-null    object        
 6   Plano           572 non-null    object        
 7   Tipo de Averia  572 non-null    object        
 8   Nodo            572 non-null    object        
dtypes: datetime64[ns](1), object(8)
memory usage: 40.3+ KB


In [35]:
df = pd.DataFrame()

In [36]:
# Aplicar y expandir resultados
cerrado_df_unique_cut[['primera_coincidencia', 'texto_match']] = cerrado_df_unique_cut['Contenido'].apply(
    lambda x: pd.Series(extraer_primer_match_detallado(x))
)

In [37]:
cerrado_df_unique_cut['Fecha de Llegada'] = cerrado_df_unique_cut['primera_coincidencia'].str.extract(r'^(\d{2}:\d*)(?=\D)')

cerrado_df_unique_cut['Fecha de Llegada'] = cerrado_df_unique_cut['Fecha de Llegada'].str.strip()
## si no hay digitos luego de los ":" completar con 0 

In [38]:
cerrado_df_unique_cut['Fecha de Llegada'] = np.where(cerrado_df_unique_cut['Fecha de Llegada'].str.len() < 4,
                                                     cerrado_df_unique_cut['Fecha de Llegada'] + '00',
                                                     cerrado_df_unique_cut['Fecha de Llegada'])

In [46]:
cerrado_df_unique_cut[cerrado_df_unique_cut.codigos_inc == 'INC000009143321']

,timestamp,sender,message,codigos_inc,Encabezado,Contenido,Plano,Tipo de Averia,Nodo,Fecha de Llegada Completa
68,2025-07-23 11:55:00,+51 914 748 998,*Se actualiza averías (Incidencias y WOs)*\nNO...,INC000009143321,*INC000009143321 PIUC109 cmtsPiura03 PARCIAL*,TECNICO PEXT: *Giuseppe Sernaque*\nEstimado at...,PIUC109,PARCIAL,cmtsPiura03,2025-07-23 11:00:00


In [47]:
cerrado_df_unique_cut.drop(['texto_match','primera_coincidencia'],axis=1,inplace=True)




KeyError: "['texto_match', 'primera_coincidencia'] not found in axis"

In [48]:
# si la hora de llegada es mayor  a la hora de envio de mensaje(cierre) entonces es de un dia antes 
hora_llegada = pd.to_datetime(cerrado_df_unique_cut['Fecha de Llegada'],format="%H:%M").dt.time

hora_timestamp = cerrado_df_unique_cut['timestamp'].dt.time

condicion = hora_timestamp < hora_llegada



# Aplicar np.where para ajustar la fecha
cerrado_df_unique_cut['Fecha de Llegada_formated'] = np.where(
    condicion,
    (cerrado_df_unique_cut['timestamp'] - pd.Timedelta(days=1)),
    cerrado_df_unique_cut['timestamp']
)
cerrado_df_unique_cut['Fecha de Llegada_formated'] = cerrado_df_unique_cut['Fecha de Llegada_formated'].dt.date

cerrado_df_unique_cut['Fecha de Llegada_formated'] = cerrado_df_unique_cut['Fecha de Llegada_formated'].astype(str)

from datetime import datetime, timedelta

cerrado_df_unique_cut['Fecha de Llegada Completa'] = pd.to_datetime(cerrado_df_unique_cut['Fecha de Llegada_formated'] +' '+cerrado_df_unique_cut['Fecha de Llegada'])

KeyError: 'Fecha de Llegada'

In [49]:
cerrado_df_unique_cut.drop(['Fecha de Llegada','Fecha de Llegada_formated'],axis=1,inplace=True)

KeyError: "['Fecha de Llegada', 'Fecha de Llegada_formated'] not found in axis"

### inspecion de resultados 

In [50]:
cerrado_df_unique_cut.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572 entries, 0 to 571
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   timestamp                  572 non-null    datetime64[ns]
 1   sender                     572 non-null    object        
 2   message                    572 non-null    object        
 3   codigos_inc                572 non-null    object        
 4   Encabezado                 572 non-null    object        
 5   Contenido                  572 non-null    object        
 6   Plano                      572 non-null    object        
 7   Tipo de Averia             572 non-null    object        
 8   Nodo                       572 non-null    object        
 9   Fecha de Llegada Completa  237 non-null    datetime64[ns]
dtypes: datetime64[ns](2), object(8)
memory usage: 44.8+ KB


In [51]:
for i in cerrado_df_unique_cut[cerrado_df_unique_cut['Fecha de Llegada Completa'].isna()].Contenido:
    print(i)

21-07 HI 07:52
Tecnico Marlon Cardenas
Estimados, atender
09:20 se observa plano arriba, se notifica a BOSF
*CERRADO* VALIDA BOSF <Se editó este mensaje.>
21-07 HI 07:52
Tecnico Marlon Cardenas
Estimados, atender
09:20 se observa plano arriba, se notifica a BOSF
*CERRADO* VALIDA BOSF <Se editó este mensaje.>
21-07 HI 07:52
Tecnico Marlon Cardenas
Estimados, atender
09:20 se observa plano arriba, se notifica a BOSF
*CERRADO* VALIDA BOSF <Se editó este mensaje.>
21-07 HI 07:52
Tecnico Marlon Cardenas
Estimados, atender
09:20 se observa plano arriba, se notifica a BOSF
*CERRADO* VALIDA BOSF <Se editó este mensaje.>
21-07 HI 07:52
Tecnico Marlon Cardenas
Estimados, atender
09:20 se observa plano arriba, se notifica a BOSF
*CERRADO* VALIDA BOSF <Se editó este mensaje.>
HI: 21/7/25 09:31
Tecnico Giuseppe Sernaque
Estimados, atender
*CERRADO* VALIDA BOSF PLANO RESTABLECIDO
HI: 21/7/25 09:31
Tecnico Giuseppe Sernaque
Estimados, atender
*CERRADO* VALIDA BOSF PLANO RESTABLECIDO
HI: 21/7/25 09:31

In [55]:
cerrado_df_unique_cut[cerrado_df_unique_cut['Fecha de Llegada Completa'].isna()][['codigos_inc','Contenido']].to_clipboard()

In [52]:
cerrado_df_e_m = cerrado_df_unique_cut

In [53]:
cerrado_df_e_m['Fecha de Incidencia'] = cerrado_df_e_m['Contenido'].str.extract(r'(?im)^(.*\bHI\b.*)$', expand=False)


### Fecha de reporte(en base al merge)

In [54]:
# tomo el INC busco en el que tiene duplicados 

In [55]:
### Fecha reporte = Fecha del primer mensaje 
### Fecha de Incidencia = Dentro del mensaje 
### Fecha de fin  de trabajo = Fecha del ultimo mensaje 

### df con los primeros mensajes por INC

In [56]:
df_inc_e_F = df_inc_e_F.rename(columns={'timestamp':'Fecha de Reporte'})

In [57]:
cerrado_df_e_m = pd.merge(cerrado_df_unique_cut,df_inc_e_F[['codigos_inc','Fecha de Reporte']],on='codigos_inc',how='left')

In [58]:
cerrado_df_e_m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572 entries, 0 to 571
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   timestamp                  572 non-null    datetime64[ns]
 1   sender                     572 non-null    object        
 2   message                    572 non-null    object        
 3   codigos_inc                572 non-null    object        
 4   Encabezado                 572 non-null    object        
 5   Contenido                  572 non-null    object        
 6   Plano                      572 non-null    object        
 7   Tipo de Averia             572 non-null    object        
 8   Nodo                       572 non-null    object        
 9   Fecha de Llegada Completa  237 non-null    datetime64[ns]
 10  Fecha de Incidencia        572 non-null    object        
 11  Fecha de Reporte           572 non-null    datetime64[ns]
dtypes: datet

In [59]:
cerrado_df_e_m[cerrado_df_e_m.codigos_inc == 'INC000009165964']

,timestamp,sender,message,codigos_inc,Encabezado,Contenido,Plano,Tipo de Averia,Nodo,Fecha de Llegada Completa,Fecha de Incidencia,Fecha de Reporte
220,2025-07-31 16:49:00,+51 914 748 998,*INC000009168013 PIUR044_cmtsPiura03 TOTAL*\nH...,INC000009165964,*INC000009165964 TRLH008 vcsTrujillo04 PARCIAL*,HI: 31/7/25 12:15\nTECNICO PEXT: *MARLON CARDE...,TRLH008,PARCIAL,vcsTrujillo04,2025-07-31 14:27:00,HI: 31/7/25 12:15,2025-07-31 12:17:00


### Fecha de Incidencia(Robusto)

In [60]:
import re
import numpy as np
cerrado_df_e_m['Fecha de Incidencia'] = cerrado_df_e_m['Contenido'].str.extract(r'(?im)^(.*\bHI\b.*)$', expand=False)


In [61]:
cerrado_df_e_m_d = cerrado_df_e_m

In [62]:
# Extraer la primera línea del texto
cerrado_df_e_m_d['Fecha de Incidencia'] = cerrado_df_e_m_d['Fecha de Incidencia'].str.replace(r'^[^\d]*(\d.*)', r'\1', regex=True) # espacios entre str de mas 
cerrado_df_e_m_d['Fecha de Incidencia'] = cerrado_df_e_m_d['Fecha de Incidencia'].str.replace(r'(?i)\bhi\b', '', regex=True).str.strip()
cerrado_df_e_m_d['Fecha de Incidencia'] = cerrado_df_e_m_d['Fecha de Incidencia'].str.replace('-','/').str.strip()


In [63]:
cerrado_df_e_m_d['Fecha de Incidencia'] = np.where(cerrado_df_e_m_d['Fecha de Incidencia'].str.len() <= 5,
                                                cerrado_df_e_m_d['Fecha de Incidencia'] + ' ' + cerrado_df_e_m_d['Fecha de Reporte'].dt.date.astype(str).str.replace('-','/'),
                                                cerrado_df_e_m_d['Fecha de Incidencia'] )
cerrado_df_e_m_d['Fecha de Incidencia'] = cerrado_df_e_m_d['Fecha de Incidencia'].str.replace(r'\s+', ' ', regex=True).str.strip() # espacios de mas

cerrado_df_e_m_d['Fecha de Incidencia'] = cerrado_df_e_m_d['Fecha de Incidencia'].str.replace('*', '',).str.strip() # espacios de mas



In [64]:
cerrado_df_e_m_d['Fecha de Incidencia'] = cerrado_df_e_m_d['Fecha de Incidencia'].astype(str)

In [65]:
from datetime import datetime
import pandas as pd
from dateutil import parser

def convertir_con_referencia(row):
    fecha_str = row['Fecha de Incidencia']
    fecha_ref = row['Fecha de Reporte']
    
    # Normalización básica
    fecha_str = fecha_str.strip().replace('.', ':').replace(' : ', ' ')
    
    formatos_con_ano = [
        "%H:%M %Y/%m/%d",
        "%Y-%m-%d %H:%M",
        "%d/%m/%Y %H:%M",
        "%d/%m/%y %H:%M",
        "%d/%m/%Y"
    ]
    
    formatos_sin_ano = [
        "%d/%m %H:%M",
        "%d/%m : %H:%M",
        "%m-%d %H:%M",
        "%H:%M %d/%m",
        "%d/%m : %H.%M",
        "%H:%M %d/%m",
        "%H: %M %d/%m",
        "%d.%m %H:%M"
    ]
    
    # Intentar con formatos que incluyen año
    for fmt in formatos_con_ano:
        try:
            return datetime.strptime(fecha_str, fmt)
        except ValueError:
            continue
    
    # Intentar con formatos sin año y agregar el año de referencia
    for fmt in formatos_sin_ano:
        try:
            parcial = datetime.strptime(fecha_str, fmt)
            return parcial.replace(year=fecha_ref.year)
        except ValueError:
            continue
    
    # Último intento: usar dateutil.parser
    try:
        return parser.parse(fecha_str, default=fecha_ref)
    except Exception:
        return pd.NaT


In [66]:
cerrado_df_e_m_d['Fecha de Incidencia_Formated'] = cerrado_df_e_m_d.apply(convertir_con_referencia, axis=1)


In [67]:
cerrado_df_e_m_d.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572 entries, 0 to 571
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   timestamp                     572 non-null    datetime64[ns]
 1   sender                        572 non-null    object        
 2   message                       572 non-null    object        
 3   codigos_inc                   572 non-null    object        
 4   Encabezado                    572 non-null    object        
 5   Contenido                     572 non-null    object        
 6   Plano                         572 non-null    object        
 7   Tipo de Averia                572 non-null    object        
 8   Nodo                          572 non-null    object        
 9   Fecha de Llegada Completa     237 non-null    datetime64[ns]
 10  Fecha de Incidencia           572 non-null    object        
 11  Fecha de Reporte              57

## Creo Tipo de Falla 
### Cuando no hay tecnico es falsa averia

In [68]:
cerrado_df_e = cerrado_df_e_m_d
##Cambiar orden primero las fehcas luego el tipo de falla


In [69]:
cerrado_df_e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572 entries, 0 to 571
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   timestamp                     572 non-null    datetime64[ns]
 1   sender                        572 non-null    object        
 2   message                       572 non-null    object        
 3   codigos_inc                   572 non-null    object        
 4   Encabezado                    572 non-null    object        
 5   Contenido                     572 non-null    object        
 6   Plano                         572 non-null    object        
 7   Tipo de Averia                572 non-null    object        
 8   Nodo                          572 non-null    object        
 9   Fecha de Llegada Completa     237 non-null    datetime64[ns]
 10  Fecha de Incidencia           572 non-null    object        
 11  Fecha de Reporte              57

In [70]:
cerrado_df_e['Tipo de falla'] = pd.NA

In [71]:
cerrado_df_e['Tipo de falla'] = np.where(
    (cerrado_df_e['Contenido'].str.count('\n') < 7)|(cerrado_df_e['Fecha de Incidencia'].isna()) ,
    'FALSA AVERIA',
    cerrado_df_e['Tipo de falla']
)

In [72]:
import numpy as np  # por completar todos los tipos de falla
#cerrado_df_e['Contenido'] = cerrado_df_e['Contenido'].str.lower()
palabras_categoria_A =  ['domicilios','clientes no cuentan con energia','fuente con normalidad','informa retorno de energia',
                         'corte de energía lado clientes','informa tener corte de energia en la zona','corte domiciliario','corte de energia lado cliente',
                        'retorna la energía comercial','clientes sin energía comercial','clientes sin energía comercial',
                         'corte de energia en la zona','NO TIENE ENERGIA ELECTRICA','corte de energia en la zona','corte de energía lado cliente','tecnico indica corte de energía',
                        'sin energía comercial','se repuso energía eléctrica','retorno de energía en la zona']
palabras_categoria_B = ['cambio','cable 500']
palabras_categoria_C = ['sin intervencion del pext','tecnico indica voltaje conforme','breaker en ON','térmica bajo'
                        ,'fuente trabajando con normalidad','se observa plano arriba','observa plano restablecido']
palabras_categoria_D = ['variación de voltaje lado clientes','cerrados en automatico']
palabras_categoria_E = ['ggee','GGEE','GG.EE','se instala GE']
palabras_categoria_F = ['nodo sin po','potencia óptica']
palabras_categoria_G = ['POR CORTE PROGRAMADO','CORTE PROGRAMDO','corte programado']
palabras_categoria_H = ['Amplificador']
palabras_categoria_I = ['TAP']


# Condiciones


condiciones = [
    cerrado_df_e['Contenido'].str.contains('|'.join(palabras_categoria_A), case=False, na=False),
    cerrado_df_e['Contenido'].str.contains('|'.join(palabras_categoria_B), case=False, na=False),
    cerrado_df_e['Contenido'].str.contains('|'.join(palabras_categoria_C), case=False, na=False),
    cerrado_df_e['Contenido'].str.contains('|'.join(palabras_categoria_D), case=False, na=False),
    cerrado_df_e['Contenido'].str.contains('|'.join(palabras_categoria_E), case=False, na=False),
    cerrado_df_e['Contenido'].str.contains('|'.join(palabras_categoria_F), case=False, na=False),
    cerrado_df_e['Contenido'].str.contains('|'.join(palabras_categoria_G), case=False, na=False),
    cerrado_df_e['Contenido'].str.contains('|'.join(palabras_categoria_H), case=False, na=False),
    cerrado_df_e['Contenido'].str.contains('|'.join(palabras_categoria_I), case=False, na=False)



]
# Resultados correspondientes
categorias = ['CORTE DE ENERGIA DOMICILIARIA', 'CABLE 500 AVERIADO','FALSA AVERIA','AUTONOMIA CONSUMIDA','CORTE DE ENERGIA COMERCIAL','CORTE DE FIBRA','CORTE PROGRAMADO','AMPLIFICADOR AVERIADO'
            ,'ELEMENTO PASIVO AVERIADO']

# Aplicar condiciones
cerrado_df_e['Tipo de falla'] = np.select(condiciones, categorias,default=cerrado_df_e['Tipo de falla'])
cerrado_df_e['Tipo de falla'] = cerrado_df_e['Tipo de falla'].str.replace('*','')

In [73]:
cerrado_df_e['Tipo de falla'] = np.where(cerrado_df_e['Contenido'].str.len() < 105,
                                       'FALSA AVERIA',
                                     cerrado_df_e['Tipo de falla'])

In [74]:
cerrado_df_e['Tipo de falla'] =  cerrado_df_e['Tipo de falla'].fillna('otro')

In [75]:
cerrado_df_e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572 entries, 0 to 571
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   timestamp                     572 non-null    datetime64[ns]
 1   sender                        572 non-null    object        
 2   message                       572 non-null    object        
 3   codigos_inc                   572 non-null    object        
 4   Encabezado                    572 non-null    object        
 5   Contenido                     572 non-null    object        
 6   Plano                         572 non-null    object        
 7   Tipo de Averia                572 non-null    object        
 8   Nodo                          572 non-null    object        
 9   Fecha de Llegada Completa     237 non-null    datetime64[ns]
 10  Fecha de Incidencia           572 non-null    object        
 11  Fecha de Reporte              57

In [76]:
cerrado_df_e[cerrado_df_e.codigos_inc == 'INC000009149326']

,timestamp,sender,message,codigos_inc,Encabezado,Contenido,Plano,Tipo de Averia,Nodo,Fecha de Llegada Completa,Fecha de Incidencia,Fecha de Reporte,Fecha de Incidencia_Formated,Tipo de falla
109,2025-07-25 14:35:00,+51 914 748 998,*Se actualiza averías (Incidencias y WOs)*\nNO...,INC000009149326,*INC000009149326 - CHI061 vcsChiclayo02 TOTAL*...,HI: 25/7/25 8:31\nTECNICO PEXT: *HENRY VIZLAO*...,-,OTRO,CHI061,2025-07-25 09:12:00,25/7/25 8:31,2025-07-25 08:36:00,2025-07-25 08:31:00,CORTE DE ENERGIA COMERCIAL


In [77]:
cerrado_df_e['Tipo de falla'].value_counts(dropna=False)

Tipo de falla
FALSA AVERIA                     267
CORTE DE ENERGIA DOMICILIARIA    107
CORTE PROGRAMADO                  96
CORTE DE ENERGIA COMERCIAL        52
CORTE DE FIBRA                    16
CABLE 500 AVERIADO                13
ELEMENTO PASIVO AVERIADO          10
otro                               5
AUTONOMIA CONSUMIDA                5
AMPLIFICADOR AVERIADO              1
Name: count, dtype: int64

### inspeccion de casos 

In [78]:
for i in cerrado_df_e[cerrado_df_e['Tipo de falla'] == 'otro'].Contenido:
    print(i)
    print('-------------')

23/07 HI: 21:20
TÉCNICO PEXT: *IVAN URQUIZA*
21:40 EN PROGRAMACION
22:23 Supervisor Roberto Guerra indica mantener en monitoreo por ser zona peligrosa
23:00 EN MONITOREO POR ZONA PELIGROSA
00:00 – 05:00 EN MONITOREO POR ZONA PELIGROSA
Plano levantando se informa a BOSF.
06:22 Se procede al *CIERRE* de averia.
-------------
02/08 HI: 00:08
TEC.PEXT: *HENRY VIZLAO*
00:30 EN PROGRAMACION
*AFECTACION DE FO ANILLO 01*
00:46 Supervisor Danny Huamanchumo informa, mantener planos en monitoreo ya que pertenecen a zona peligrosa
01:00-06:00 PLANOS EN MONITOREO, ZONA PELIGROSA, SE ATENDERA DURANTE EL DIA
05:45 Tecnico en breve atendera
06:41 Tecnico en plano CHI065
06:45 Tecnico informa que dorsal se encuentra en zona realizando trabajos. CHI065
07:00-08:00 PEXT y personal de FO realizando trabajos en conjunto
10:00 plano CHI055 se verifica restablecido.
11:35 Se verifica en gestores planos UP solucion brindada por empalme de FO
*CERRADO BOSF*
-------------
02/08 HI: 00:08
TEC.PEXT: *HENRY VIZLAO

In [79]:
cerrado_df_e.drop(['message','Encabezado','Fecha de Incidencia'],axis=1,inplace=True) 
cerrado_df_e.rename(columns={'timestamp' : 'Fecha Fin de Trabajo','Contenido':'Detalle','Fecha de Incidencia_Formated':'Fecha de Incidencia',
                             'Fecha de Llegada Completa':'Fecha de Llegada'},inplace=True)


## Agrego las filas al existente

In [80]:
cerrado_df_e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572 entries, 0 to 571
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Fecha Fin de Trabajo  572 non-null    datetime64[ns]
 1   sender                572 non-null    object        
 2   codigos_inc           572 non-null    object        
 3   Detalle               572 non-null    object        
 4   Plano                 572 non-null    object        
 5   Tipo de Averia        572 non-null    object        
 6   Nodo                  572 non-null    object        
 7   Fecha de Llegada      237 non-null    datetime64[ns]
 8   Fecha de Reporte      572 non-null    datetime64[ns]
 9   Fecha de Incidencia   572 non-null    datetime64[ns]
 10  Tipo de falla         572 non-null    object        
dtypes: datetime64[ns](4), object(7)
memory usage: 49.3+ KB


In [81]:
def proces_old_data(df_old,df_actual):
    if any(df_old):
        set_a = set(cerrado_df_e.codigos_inc.tolist())
        set_b = set(df_old.codigos_inc.tolist())
        elementos_comunes = set_a.intersection(set_b)
        if any(elementos_comunes):
            print("Incidencias duplicadas")
            df_old = df_old[~df_old.codigos_inc.isin(elementos_comunes)]
        else:
            print("sin incidencias duplicadas,agrego")
            pass 
        df_total = pd.concat([df_old,cerrado_df_e],axis=0,ignore_index = True)
    else: 
        df_total = df_actual
    return df_total

In [82]:
df_total = cerrado_df_e.copy()

# Transformacion de Data final

In [83]:
df_total['Lugar'] = df_total.Nodo.str.replace(r'^(vcs|olt|cmts)', '', regex=True)
df_total.Lugar = df_total.Lugar.str.replace('*','')
df_total.Lugar  = df_total.Lugar.str.replace(r'\d+$', '', regex=True)


In [84]:
df_total['Lugar'] = np.where(df_total['Lugar'].str.contains('chiclayo', case=False), 'Chiclayo',
                    np.where(df_total['Lugar'].str.contains('trujill', case=False), 'Trujillo',
                    np.where(df_total['Lugar'].str.contains('cajamarca', case=False), 'Cajamarca', 
                    np.where(df_total['Lugar'].str.contains('chimbote', case=False), 'Chimbote',
                    np.where(df_total['Lugar'].str.contains('sullana', case=False), 'Sullana',df_total['Lugar'])))))


In [85]:
df_total.Lugar  = df_total.Lugar.str.replace('CHI', 'Chiclayo')
df_total.Lugar  = df_total.Lugar.str.replace('TRLE', 'Trujillo')


In [86]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572 entries, 0 to 571
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Fecha Fin de Trabajo  572 non-null    datetime64[ns]
 1   sender                572 non-null    object        
 2   codigos_inc           572 non-null    object        
 3   Detalle               572 non-null    object        
 4   Plano                 572 non-null    object        
 5   Tipo de Averia        572 non-null    object        
 6   Nodo                  572 non-null    object        
 7   Fecha de Llegada      237 non-null    datetime64[ns]
 8   Fecha de Reporte      572 non-null    datetime64[ns]
 9   Fecha de Incidencia   572 non-null    datetime64[ns]
 10  Tipo de falla         572 non-null    object        
 11  Lugar                 572 non-null    object        
dtypes: datetime64[ns](4), object(8)
memory usage: 53.8+ KB


In [87]:
df_total.Lugar.value_counts()

Lugar
Trujillo     237
Chiclayo     151
Piura         62
Cajamarca     61
Chimbote      54
Sullana        7
Name: count, dtype: int64

## Calculo de columnas calculadas 

In [88]:
df_total['Tiempo de atencion'] = df_total['Fecha Fin de Trabajo'] - df_total['Fecha de Llegada']

In [89]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572 entries, 0 to 571
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype          
---  ------                --------------  -----          
 0   Fecha Fin de Trabajo  572 non-null    datetime64[ns] 
 1   sender                572 non-null    object         
 2   codigos_inc           572 non-null    object         
 3   Detalle               572 non-null    object         
 4   Plano                 572 non-null    object         
 5   Tipo de Averia        572 non-null    object         
 6   Nodo                  572 non-null    object         
 7   Fecha de Llegada      237 non-null    datetime64[ns] 
 8   Fecha de Reporte      572 non-null    datetime64[ns] 
 9   Fecha de Incidencia   572 non-null    datetime64[ns] 
 10  Tipo de falla         572 non-null    object         
 11  Lugar                 572 non-null    object         
 12  Tiempo de atencion    237 non-null    timedelta64[ns]
dtypes: da

In [90]:
df_total = df_total[['sender','codigos_inc','Plano','Lugar','Tipo de Averia','Tipo de falla','Nodo','Fecha de Incidencia','Fecha de Llegada','Tiempo de atencion',
                                             'Fecha Fin de Trabajo','Fecha de Reporte','Detalle']]

In [91]:
try: 
    #df_old = pd.read_excel(r'D:\DATA\Test_watchdog\Resultados\test_v0.xlsx')
    df_old = pd.read_excel(r'Resultados\Averias-HFC.xlsx')
    
except:
    print("no existe archivo creado,creo uno")
    df_old = pd.DataFrame()
df_total = proces_old_data(df_old,df_total)

no existe archivo creado,creo uno


# Cargo el resultado

In [60]:
df_total.to_excel(r'Resultados\Averias-HFC.xlsx',index=False)
#df_total.to_excel(r'D:\DATA\Test_watchdog\Resultados\test_v0.xlsx',index=False)

In [85]:
#df_total.to_csv(r'D:\DATA\Test_watchdog\Resultados\test_v0.xlsx',index=False)